## Correlation matrices and eigenvalues of merged per-core logs:

In [22]:
import os
import pandas as pd
import numpy as np
from numpy.linalg import eig, eigvalsh

import eigen_analysis

In [23]:
def get_latencies(out_fname):
    with open(out_fname, 'r') as f:
        lines = f.readlines()
    header = lines[0].rstrip('\n').split()
    read_lat = lines[1].rstrip('\n').split()
    lat = {'read': dict(zip(header[1:], [float(y) for y in read_lat[1:]]))}

    return lat['read']

In [24]:
def get_joules(df):
    joules_sum = df['joules_diff'].sum()
    joules = {'joules_sum': joules_sum}
    
    return joules

### Correlation Matrices of (instructions_diff, cycles_diff, ref_cycles_diff, llc_miss_diff):

In [28]:
corr_dvfs_counter_list = {}
eig_dvfs_counters_list = {}
corr_dvfs_non_counter_list = {}
eig_dvfs_non_counters_list = {}
joules_dvfs_list = {}

for dvfs in ['0xd00', '0xf00', '0x1100', '0x1300', '0x1500', '0x1700', '0x1900', '0x1b00', '0x1d00']:
    corr_qps_counter_list = {}
    corr_qps_non_counter_list = {}
    joules_qps_list = {}
    eig_qps_counters_list = {}
    eig_qps_non_counters_list = {}
    
    for qps in ['200k', '400k', '600k']:
        corr_itr_counter_list = {}
        corr_itr_non_counter_list = {}
        joules_itr_list = {}
        eig_itr_counters_list = {}
        eig_itr_non_counters_list = {}
        
        all_itr_counter_dir = qps + '_' + dvfs + '_counters_merged/'
        all_itr_non_counter_dir = qps + '_' + dvfs + '_non_counters_merged/'

        print('Parsing counter metrics in ' + all_itr_counter_dir)
        for file in os.listdir(all_itr_counter_dir):
            if (os.path.isdir(all_itr_non_counter_dir + file) or (file == '.gitignore')):
                continue
            itr = file.split('_')[2]
            df = pd.read_csv(all_itr_counter_dir + file, sep = ',')
            joules = get_joules(df.copy())
            joules_itr_list[itr] = joules
            print('ITR_DELAY: ', itr, joules)
            df = df.drop([df.columns[0], 'joules_diff', 'timestamp'], axis=1)
            df_corr = df.corr()
            corr_itr_counter_list[itr] = df_corr
            
            vals_counters_dict = {}
            vals_counters, vecs_counter = eig(df_corr)
            i = 0
            for val in vals_counters:
                if val <= 0:
                    print('EIGENVAL <= 0')
                vals_counters_dict['eig_' + str(i)] = val
                i += 1
            eig_itr_counters_list[itr] = vals_counters_dict
                
        print('Parsing non-counter metrics in ' + all_itr_non_counter_dir)
        for file in os.listdir(all_itr_non_counter_dir):
            if (os.path.isdir(all_itr_non_counter_dir + file) or (file == '.gitignore')):
                continue
            itr = file.split('_')[2]
            print('ITR_DELAY: ', itr)
            df = pd.read_csv(all_itr_non_counter_dir + file, sep= ',')
            df_corr = df.drop([df.columns[0], 'timestamp'], axis = 1).corr()
            corr_itr_non_counter_list[itr] = df_corr
            
            i = 4
            vals_non_counters_dict = {}
            vals_non_counters, vecs_non_counter = eig(df_corr)
            for val in vals_non_counters:
                if val <= 0:
                    print('EIGENVAL <= 0')
                vals_non_counters_dict['eig_' + str(i)] = val
                i += 1
            eig_itr_non_counters_list[itr] = vals_non_counters_dict
            
        corr_qps_counter_list[qps] = corr_itr_counter_list
        corr_qps_non_counter_list[qps] = corr_itr_non_counter_list
        joules_qps_list[qps] = joules_itr_list
        eig_qps_counters_list[qps] = eig_itr_counters_list
        eig_qps_non_counters_list[qps] = eig_itr_non_counters_list
        
    corr_dvfs_counter_list[dvfs] = corr_qps_counter_list
    corr_dvfs_non_counter_list[dvfs] = corr_qps_non_counter_list
    joules_dvfs_list[dvfs] = joules_qps_list
    eig_dvfs_counters_list[dvfs] = eig_qps_counters_list
    eig_dvfs_non_counters_list[dvfs] = eig_qps_non_counters_list

Parsing counter metrics in 200k_0xd00_counters_merged/
ITR_DELAY:  40 {'joules_sum': 35745.28953299998}
ITR_DELAY:  200 {'joules_sum': 34341.416562000035}
ITR_DELAY:  20 {'joules_sum': 35966.11422999994}
ITR_DELAY:  400 {'joules_sum': 33724.28090399999}
ITR_DELAY:  2 {'joules_sum': 36046.76196400005}
ITR_DELAY:  30 {'joules_sum': 35869.15314400006}
ITR_DELAY:  350 {'joules_sum': 34207.359545999956}
ITR_DELAY:  50 {'joules_sum': 35496.66981999989}
ITR_DELAY:  10 {'joules_sum': 36553.16609099985}
ITR_DELAY:  100 {'joules_sum': 34939.679369000085}
ITR_DELAY:  250 {'joules_sum': 34107.859213000156}
ITR_DELAY:  300 {'joules_sum': 34050.92852300004}
Parsing non-counter metrics in 200k_0xd00_non_counters_merged/
ITR_DELAY:  10
ITR_DELAY:  40
ITR_DELAY:  400
ITR_DELAY:  300
ITR_DELAY:  50
ITR_DELAY:  20
ITR_DELAY:  350
ITR_DELAY:  250
ITR_DELAY:  200
ITR_DELAY:  100
ITR_DELAY:  2
ITR_DELAY:  30
Parsing counter metrics in 400k_0xd00_counters_merged/
ITR_DELAY:  300 {'joules_sum': 37753.34609300

ITR_DELAY:  50 {'joules_sum': 45022.95368800004}
ITR_DELAY:  300 {'joules_sum': 42346.84958899999}
ITR_DELAY:  10 {'joules_sum': 46209.43247999999}
ITR_DELAY:  20 {'joules_sum': 45603.053562000016}
ITR_DELAY:  40 {'joules_sum': 45269.76554399982}
ITR_DELAY:  2 {'joules_sum': 45837.75941899989}
ITR_DELAY:  100 {'joules_sum': 44286.287431999925}
ITR_DELAY:  200 {'joules_sum': 42870.843736999996}
ITR_DELAY:  250 {'joules_sum': 42566.03674100002}
ITR_DELAY:  400 {'joules_sum': 42026.18674100013}
Parsing non-counter metrics in 200k_0x1300_non_counters_merged/
ITR_DELAY:  20
ITR_DELAY:  10
ITR_DELAY:  400
ITR_DELAY:  50
ITR_DELAY:  300
ITR_DELAY:  100
ITR_DELAY:  30
ITR_DELAY:  200
ITR_DELAY:  250
ITR_DELAY:  40
ITR_DELAY:  350
ITR_DELAY:  2
Parsing counter metrics in 400k_0x1300_counters_merged/
ITR_DELAY:  100 {'joules_sum': 49650.52428100002}
ITR_DELAY:  30 {'joules_sum': 52333.21461099989}
ITR_DELAY:  20 {'joules_sum': 52576.62638800002}
ITR_DELAY:  200 {'joules_sum': 47881.12942800006}


ITR_DELAY:  10 {'joules_sum': 58751.10055600022}
ITR_DELAY:  40 {'joules_sum': 57962.139365999996}
ITR_DELAY:  400 {'joules_sum': 51904.72810200011}
ITR_DELAY:  100 {'joules_sum': 55983.79755299978}
ITR_DELAY:  250 {'joules_sum': 53623.066503999944}
ITR_DELAY:  50 {'joules_sum': 57481.28569899997}
ITR_DELAY:  2 {'joules_sum': 59213.14426299997}
Parsing non-counter metrics in 200k_0x1900_non_counters_merged/
ITR_DELAY:  50
ITR_DELAY:  10
ITR_DELAY:  300
ITR_DELAY:  2
ITR_DELAY:  100
ITR_DELAY:  20
ITR_DELAY:  350
ITR_DELAY:  250
ITR_DELAY:  200
ITR_DELAY:  40
ITR_DELAY:  400
ITR_DELAY:  30
Parsing counter metrics in 400k_0x1900_counters_merged/
ITR_DELAY:  10 {'joules_sum': 67931.10407699997}
ITR_DELAY:  30 {'joules_sum': 66606.38842700003}
ITR_DELAY:  400 {'joules_sum': 57570.191307999805}
ITR_DELAY:  250 {'joules_sum': 58929.76748200001}
ITR_DELAY:  350 {'joules_sum': 59241.25531999988}
ITR_DELAY:  20 {'joules_sum': 67700.3283279998}
ITR_DELAY:  100 {'joules_sum': 62829.54713599995}
I

In [29]:
# checking corr matrix
corr_dvfs_counter_list['0x1100']['200k']['300']

,instructions_diff,cycles_diff,ref_cycles_diff,llc_miss_diff
instructions_diff,1.000000,0.890231,0.876964,0.760093
cycles_diff,0.890231,1.000000,0.987090,0.824369
ref_cycles_diff,0.876964,0.987090,1.000000,0.827097
llc_miss_diff,0.760093,0.824369,0.827097,1.000000


In [30]:
# checking corr matrix
corr_dvfs_non_counter_list['0x1900']['600k']['100']

,rx_bytes,rx_desc,tx_bytes,tx_desc
rx_bytes,1.000000,0.675435,0.007237,0.006457
rx_desc,0.675435,1.000000,0.025279,0.018602
tx_bytes,0.007237,0.025279,1.000000,0.551350
tx_desc,0.006457,0.018602,0.551350,1.000000


In [31]:
# checking joules sum values
joules_dvfs_list['0x1300']['400k']

{'100': {'joules_sum': 49650.52428100002},
 '30': {'joules_sum': 52333.21461099989},
 '20': {'joules_sum': 52576.62638800002},
 '200': {'joules_sum': 47881.12942800006},
 '400': {'joules_sum': 45767.233109999914},
 '300': {'joules_sum': 45941.77930299987},
 '40': {'joules_sum': 51859.58621099981},
 '350': {'joules_sum': 45848.74844700005},
 '250': {'joules_sum': 47282.107658999965},
 '10': {'joules_sum': 53032.624872999964},
 '2': {'joules_sum': 53097.84784200022},
 '50': {'joules_sum': 50887.24218500004}}

In [32]:
eig_dvfs_counters_list['0x1900']['400k']

{'10': {'eig_0': 3.4372564430776453,
  'eig_1': 0.22825471464794028,
  'eig_2': 0.33448722689752125,
  'eig_3': 1.6153768934222783e-06},
 '30': {'eig_0': 3.468688416697106,
  'eig_1': 0.21927242031201044,
  'eig_2': 0.3120320109996304,
  'eig_3': 7.151991250479851e-06},
 '400': {'eig_0': 3.5202200536419532,
  'eig_1': 0.35910315143470933,
  'eig_2': 0.08965273316601653,
  'eig_3': 0.031024061757321246},
 '250': {'eig_0': 3.562775555177687,
  'eig_1': 0.2926878835097507,
  'eig_2': 0.10224382066713489,
  'eig_3': 0.04229274064542779},
 '350': {'eig_0': 3.7478649772718615,
  'eig_1': 0.18173746579816039,
  'eig_2': 0.053444424859359234,
  'eig_3': 0.01695313207062041},
 '20': {'eig_0': 3.8926509386479893,
  'eig_1': 0.06614636768087469,
  'eig_2': 0.04120138577054526,
  'eig_3': 1.3079005853034056e-06},
 '100': {'eig_0': 3.6257536248513547,
  'eig_1': 0.23402733090557917,
  'eig_2': 0.12720407506275688,
  'eig_3': 0.013014969180309536},
 '2': {'eig_0': 3.4776699593129266,
  'eig_1': 0.18

In [33]:
eig_dvfs_non_counters_list['0x1300']['400k']

{'300': {'eig_4': 1.715516499031768,
  'eig_5': 1.5962605076090093,
  'eig_6': 0.29637991839131206,
  'eig_7': 0.39184307496791193},
 '100': {'eig_4': 1.6634045330823812,
  'eig_5': 1.5367612777189033,
  'eig_6': 0.34139839430326197,
  'eig_7': 0.45843579489545466},
 '400': {'eig_4': 1.7756790271055451,
  'eig_5': 1.5673775235306417,
  'eig_6': 0.2973432419570651,
  'eig_7': 0.3596002074067468},
 '10': {'eig_4': 2.1585601056321613,
  'eig_5': 0.9733166682174103,
  'eig_6': 0.29977939327076086,
  'eig_7': 0.5683438328796685},
 '250': {'eig_4': 1.6995994781395218,
  'eig_5': 1.5840968931759043,
  'eig_6': 0.30655676702305384,
  'eig_7': 0.4097468616615206},
 '30': {'eig_4': 1.7874663103429433,
  'eig_5': 1.3225330442283585,
  'eig_6': 0.3610814999127617,
  'eig_7': 0.5289191455159359},
 '200': {'eig_4': 1.6864088057431337,
  'eig_5': 1.573685122873135,
  'eig_6': 0.31756789450837636,
  'eig_7': 0.42233817687535374},
 '50': {'eig_4': 1.669989657391513,
  'eig_5': 1.4781364942271111,
  'ei

### Latency Measurements:

In [34]:
lat_dvfs_list = {}
for dvfs in ['0xd00', '0xf00', '0x1100', '0x1300', '0x1500', '0x1700', '0x1900', '0x1b00', '0x1d00']:
    lat_qps_list = {}
    for qps in ['200k', '400k', '600k']:
        lat_itr_list = {}
        lat_dir = qps + '_qps/linux_mcd_out_0_' + dvfs + '_135_' + qps + '/'

        for itr in corr_dvfs_counter_list[dvfs][qps].keys():
            lat_file = lat_dir + 'linux.mcd.out.0_' + str(itr) + '_' + dvfs + '_135_' + qps[:-1] + '000'
            latencies = get_latencies(lat_file)
            lat_itr_list[itr] = latencies
        lat_qps_list[qps] = lat_itr_list
    lat_dvfs_list[dvfs] = lat_qps_list

In [35]:
lat_dvfs_list['0x1900']['200k']['100']

{'avg': 101.6,
 'std': 30.9,
 'min': 37.4,
 '5th': 53.2,
 '10th': 59.2,
 '50th': 101.9,
 '90th': 142.1,
 '95th': 151.6,
 '99th': 170.1}

### Putting it all together:

In [36]:
lat_columns = []
for itr in [list(lat_dvfs_list['0x1100']['200k'].keys())[0]]:
    lat_columns = list(lat_dvfs_list['0x1100']['200k'][itr].keys())
lat_columns

['avg', 'std', 'min', '5th', '10th', '50th', '90th', '95th', '99th']

In [37]:
eig_counters_columns = []
for itr in [list(eig_dvfs_counters_list['0x1100']['200k'].keys())[0]]:
    eig_counters_columns = list(eig_dvfs_counters_list['0x1100']['200k'][itr].keys())
eig_non_counters_columns = []
for itr in [list(eig_dvfs_non_counters_list['0x1100']['200k'].keys())[0]]:
    eig_non_counters_columns += list(eig_dvfs_non_counters_list['0x1100']['200k'][itr].keys())
eig_columns = eig_counters_columns + eig_non_counters_columns
eig_columns

['eig_0', 'eig_1', 'eig_2', 'eig_3', 'eig_4', 'eig_5', 'eig_6', 'eig_7']

In [38]:
all_cols = lat_columns + eig_columns + ['joules_sum']
all_cols.append('desc')
print(all_cols)

['avg', 'std', 'min', '5th', '10th', '50th', '90th', '95th', '99th', 'eig_0', 'eig_1', 'eig_2', 'eig_3', 'eig_4', 'eig_5', 'eig_6', 'eig_7', 'joules_sum', 'desc']


In [39]:
eig_dvfs_non_counters_list['0x1300']['200k']['10']

{'eig_4': 2.294147382104687,
 'eig_5': 0.9014327690853716,
 'eig_6': 0.5649305538527354,
 'eig_7': 0.23948929495720625}

In [42]:
eig_lat_joules_df = pd.DataFrame(columns=all_cols)
for dvfs in ['0xd00', '0xf00', '0x1100', '0x1300', '0x1500', '0x1700', '0x1900', '0x1b00', '0x1d00']:
    for qps in ['200k', '400k', '600k']:
        for itr in corr_dvfs_counter_list[dvfs][qps].keys():
            desc = '0' + '_' + itr + '_' + dvfs + '_' + '135' + '_' + qps
            new_row = {}
            for p in lat_columns:
                new_row[p] = lat_dvfs_list[dvfs][qps][itr][p]
            for e in eig_counters_columns:
                new_row[e] = eig_dvfs_counters_list[dvfs][qps][itr][e]
            for e in eig_non_counters_columns:
                new_row[e] = eig_dvfs_non_counters_list[dvfs][qps][itr][e]
            new_row['joules_sum'] = joules_dvfs_list[dvfs][qps][itr]['joules_sum']
            new_row['desc'] = desc
            eig_lat_joules_df = eig_lat_joules_df._append(new_row, ignore_index = True)

In [43]:
eig_lat_joules_df

,avg,std,min,5th,10th,50th,90th,95th,99th,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,eig_7,joules_sum,desc
0,81.8,17.0,45.3,58.8,61.6,79.8,104.5,112.7,130.8,3.492777,0.199347,0.307877,1.926252e-08,1.874681,1.206367,0.388455,0.530496,35745.289533,0_40_0xd00_135_200k
1,179.3,57.8,45.3,86.6,100.6,180.9,256.3,270.5,297.3,3.608221,0.258202,0.133167,4.096448e-04,1.670789,1.540596,0.354743,0.433872,34341.416562,0_200_0xd00_135_200k
2,74.0,14.6,49.8,57.1,59.7,71.1,92.8,102.1,124.4,3.524457,0.190145,0.285398,2.713862e-08,2.161959,0.972311,0.308426,0.557303,35966.114230,0_20_0xd00_135_200k
3,292.5,113.5,49.8,111.7,135.7,294.9,442.9,471.3,515.1,3.593040,0.285410,0.121201,3.490791e-04,1.696883,1.574022,0.327540,0.401555,33724.280904,0_400_0xd00_135_200k
4,71.3,17.4,41.1,55.1,56.2,66.2,93.0,104.9,134.4,3.599618,0.245315,0.155067,8.947988e-08,2.069599,1.061065,0.347774,0.521561,36046.761964,0_2_0xd00_135_200k
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,57.5,17.2,34.0,42.0,43.3,53.5,75.6,86.5,123.3,3.956992,0.026499,0.016508,1.003158e-06,2.014633,1.106153,0.334646,0.544568,92513.073726,0_2_0x1d00_135_600k
320,206.0,71.4,41.1,92.7,110.4,205.8,300.6,322.0,358.2,3.423851,0.406464,0.125272,4.441266e-02,1.707851,1.596945,0.296460,0.398744,74429.830394,0_250_0x1d00_135_600k
321,71.1,14.7,34.0,49.2,52.7,71.0,88.0,94.9,110.9,3.879166,0.084843,0.035979,1.202505e-05,1.657982,1.498096,0.350948,0.492973,86921.377698,0_40_0x1d00_135_600k
322,106.0,29.2,37.4,60.0,66.7,106.3,145.2,152.8,168.8,3.513134,0.362429,0.120706,3.730351e-03,1.677405,1.543122,0.327947,0.451526,81377.862032,0_100_0x1d00_135_600k


In [47]:
outdir = 'eig_lat_joules_dfs/'
eig_lat_joules_df.to_csv(outdir + 'all_dvfs_qps_itr')

In [48]:
df = pd.read_csv(outdir + 'all_dvfs_qps_itr', index_col=0)

In [49]:
df

,avg,std,min,5th,10th,50th,90th,95th,99th,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,eig_7,joules_sum,desc
0,81.8,17.0,45.3,58.8,61.6,79.8,104.5,112.7,130.8,3.492777,0.199347,0.307877,1.926252e-08,1.874681,1.206367,0.388455,0.530496,35745.289533,0_40_0xd00_135_200k
1,179.3,57.8,45.3,86.6,100.6,180.9,256.3,270.5,297.3,3.608221,0.258202,0.133167,4.096448e-04,1.670789,1.540596,0.354743,0.433872,34341.416562,0_200_0xd00_135_200k
2,74.0,14.6,49.8,57.1,59.7,71.1,92.8,102.1,124.4,3.524457,0.190145,0.285398,2.713862e-08,2.161959,0.972311,0.308426,0.557303,35966.114230,0_20_0xd00_135_200k
3,292.5,113.5,49.8,111.7,135.7,294.9,442.9,471.3,515.1,3.593040,0.285410,0.121201,3.490791e-04,1.696883,1.574022,0.327540,0.401555,33724.280904,0_400_0xd00_135_200k
4,71.3,17.4,41.1,55.1,56.2,66.2,93.0,104.9,134.4,3.599618,0.245315,0.155067,8.947988e-08,2.069599,1.061065,0.347774,0.521561,36046.761964,0_2_0xd00_135_200k
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,57.5,17.2,34.0,42.0,43.3,53.5,75.6,86.5,123.3,3.956992,0.026499,0.016508,1.003158e-06,2.014633,1.106153,0.334646,0.544568,92513.073726,0_2_0x1d00_135_600k
320,206.0,71.4,41.1,92.7,110.4,205.8,300.6,322.0,358.2,3.423851,0.406464,0.125272,4.441266e-02,1.707851,1.596945,0.296460,0.398744,74429.830394,0_250_0x1d00_135_600k
321,71.1,14.7,34.0,49.2,52.7,71.0,88.0,94.9,110.9,3.879166,0.084843,0.035979,1.202505e-05,1.657982,1.498096,0.350948,0.492973,86921.377698,0_40_0x1d00_135_600k
322,106.0,29.2,37.4,60.0,66.7,106.3,145.2,152.8,168.8,3.513134,0.362429,0.120706,3.730351e-03,1.677405,1.543122,0.327947,0.451526,81377.862032,0_100_0x1d00_135_600k
